# Match Data Master Table Creation

This notebook combines match statistics and fixtures data to create the foundation of a master table for football prediction modeling.

## Objectives:
- Load and explore match statistics data
- Load and explore fixtures data
- Understand the relationship between both datasets
- Design and create the first version of a master table
- Ensure data quality and consistency

In [1]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up paths
project_root = Path().resolve().parent.parent.parent
data_dev_path = project_root / 'data' / 'dev' / 'raw'
data_prod_path = project_root / 'data' / 'prod' / 'raw'

print(f"Project root: {project_root}")
print(f"Dev data path: {data_dev_path}")
print(f"Prod data path: {data_prod_path}")

c:\Users\50230\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Project root: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football
Dev data path: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\dev\raw
Prod data path: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw


## Data Discovery

Let's first identify what match stats and fixtures files are available in our data directories.

## Load Match Statistics Data

Let's load the most comprehensive match statistics dataset available.


In [2]:
def find_data_files(path, keywords):
    """Find files containing specific keywords in their names"""
    files = []
    if path.exists():
        for file in os.listdir(path):
            if any(keyword.lower() in file.lower() for keyword in keywords):
                files.append(file)
    return files

# Look for match stats files
match_stats_keywords = ['match_stats', 'stats']
fixtures_keywords = ['fixtures', 'fixture']

print("=== MATCH STATS FILES ===")
print("\nDev environment:")
dev_stats_files = find_data_files(data_dev_path, match_stats_keywords)
for file in dev_stats_files:
    print(f"  - {file}")

print("\nProd environment:")
prod_stats_files = find_data_files(data_prod_path, match_stats_keywords)
for file in prod_stats_files:
    print(f"  - {file}")

# Check match_stats subdirectory
match_stats_dir_dev = data_dev_path / 'match_stats'
match_stats_dir_prod = data_prod_path / 'match_stats'

if match_stats_dir_dev.exists():
    print("\nDev match_stats directory:")
    stats_subfiles = os.listdir(match_stats_dir_dev)
    print(f"  Found {len(stats_subfiles)} files")
    for file in stats_subfiles[:5]:  # Show first 5
        print(f"  - {file}")
    if len(stats_subfiles) > 5:
        print(f"  ... and {len(stats_subfiles) - 5} more files")

if match_stats_dir_prod.exists():
    print("\nProd match_stats directory:")
    stats_subfiles = os.listdir(match_stats_dir_prod)
    print(f"  Found {len(stats_subfiles)} files")
    for file in stats_subfiles[:5]:  # Show first 5
        print(f"  - {file}")
    if len(stats_subfiles) > 5:
        print(f"  ... and {len(stats_subfiles) - 5} more files")

print("\n=== FIXTURES FILES ===")
print("\nDev environment:")
dev_fixtures_files = find_data_files(data_dev_path, fixtures_keywords)
for file in dev_fixtures_files:
    print(f"  - {file}")

print("\nProd environment:")
prod_fixtures_files = find_data_files(data_prod_path, fixtures_keywords)
for file in prod_fixtures_files:
    print(f"  - {file}")

=== MATCH STATS FILES ===

Dev environment:
  - match_stats
  - match_statsarsenal_premier_league_2023_2024_long_format_sample.json
  - match_stats_newcastle_arsenal.json

Prod environment:
  - match_stats

Dev match_stats directory:
  Found 31 files
  - arsenal_premier_league_2023_2024_long_format_sample.json
  - progress_match_stats_10.json
  - progress_match_stats_100.json
  - progress_match_stats_110.json
  - progress_match_stats_120.json
  ... and 26 more files

Prod match_stats directory:
  Found 1 files
  - all_match_stats.json

=== FIXTURES FILES ===

Dev environment:
  - all_competitions_fixtures_2019_2024.json
  - all_competitions_fixtures_2019_2025.json
  - premier_league_fixtures_2019_2024.json

Prod environment:
  - all_competitions_fixtures.json
  - all_competitions_fixtures_dataframe.csv
  - all_competitions_fixtures_dataframe.json


In [3]:
def load_json_data(file_path):
    """Load JSON data with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Successfully loaded: {file_path}")
        return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Try to load comprehensive match stats data
match_stats_df = None

# Priority order: prod comprehensive file, then dev files
potential_stats_files = [
    (data_prod_path / 'match_stats' / 'all_match_stats.json', 'Prod all match stats'),
    (data_dev_path / 'match_stats' / 'all_match_stats.json', 'Dev all match stats'),
]

# Add individual progress files from dev if comprehensive not available
if match_stats_dir_dev.exists():
    progress_files = [f for f in os.listdir(match_stats_dir_dev) if f.startswith('progress_match_stats')]
    if progress_files:
        # Try the largest progress file (highest number)
        progress_numbers = []
        for f in progress_files:
            try:
                num = int(f.split('_')[-1].split('.')[0])
                progress_numbers.append((num, f))
            except:
                pass
        if progress_numbers:
            progress_numbers.sort(reverse=True)
            largest_file = progress_numbers[0][1]
            potential_stats_files.append(
                (match_stats_dir_dev / largest_file, f'Dev progress file: {largest_file}')
            )

print("=== Loading Match Statistics Data ===")
for file_path, description in potential_stats_files:
    if file_path.exists():
        print(f"\nTrying {description}: {file_path}")
        stats_data = load_json_data(file_path)
        
        if stats_data:
            # Check if it's a list or dict
            if isinstance(stats_data, list):
                match_stats_df = pd.DataFrame(stats_data)
            elif isinstance(stats_data, dict) and len(stats_data) > 0:
                # Try to convert dict to DataFrame
                try:
                    match_stats_df = pd.DataFrame([stats_data])
                except:
                    print(f"Could not convert dict to DataFrame")
                    continue
            
            if match_stats_df is not None and len(match_stats_df) > 0:
                print(f"Successfully loaded match stats: {match_stats_df.shape}")
                break
            else:
                print(f"Empty or invalid data in {file_path}")
                match_stats_df = None
    else:
        print(f"File not found: {file_path}")

if match_stats_df is None:
    print("\nNo match statistics data could be loaded!")
else:
    print(f"\nFinal match stats DataFrame shape: {match_stats_df.shape}")
    print(f"Columns: {list(match_stats_df.columns)}")

=== Loading Match Statistics Data ===

Trying Prod all match stats: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\match_stats\all_match_stats.json
Successfully loaded: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\match_stats\all_match_stats.json
Successfully loaded match stats: (98164, 4)

Final match stats DataFrame shape: (98164, 4)
Columns: ['match_id', 'team_name', 'stat_name', 'stat_value']


## Load Team Mapping Data

Let's load the team mapping information that contains the standardized team names and team IDs."

In [4]:
# Try to load the most promising team mapping file
team_mapping_df = None

# Priority order for team mapping files
potential_mapping_files = [
    (data_prod_path / 'all_teams.json', 'Prod all teams'),
    (data_dev_path / 'all_teams.json', 'Dev all teams'),
]

print("=== LOADING TEAM MAPPING DATA ===")

for file_path, description in potential_mapping_files:
    if file_path.exists():
        print(f"\nTrying {description}: {file_path}")
        team_mapping_data = load_json_data(file_path)
        
        if team_mapping_data:
            # Check if it's a list or dict
            if isinstance(team_mapping_data, list):
                team_mapping_df = pd.DataFrame(team_mapping_data)
            elif isinstance(team_mapping_data, dict):
                # Could be nested structure, try to flatten
                try:
                    # If dict contains lists, try to extract them
                    if any(isinstance(v, list) for v in team_mapping_data.values()):
                        # Find the largest list (likely the main data)
                        max_key = max(team_mapping_data.keys(), key=lambda k: len(team_mapping_data[k]) if isinstance(team_mapping_data[k], list) else 0)
                        team_mapping_df = pd.DataFrame(team_mapping_data[max_key])
                    else:
                        team_mapping_df = pd.DataFrame([team_mapping_data])
                except Exception as e:
                    print(f"Error processing team mapping data: {e}")
                    continue
            
            if team_mapping_df is not None and len(team_mapping_df) > 0:
                print(f"Successfully loaded team mapping: {team_mapping_df.shape}")
                break
            else:
                print(f"Empty or invalid team mapping data")
                team_mapping_df = None
    else:
        print(f"File not found: {file_path}")

if team_mapping_df is None:
    print("\nNo team mapping data could be loaded!")
    print("Let's check if we can extract team mapping from fixtures data...")
    
    # Create a basic team mapping from fixtures data
    if fixtures_df is not None:
        team_mapping_df = fixtures_df[['team_name', 'team_id']].drop_duplicates().reset_index(drop=True)
        print(f"Created basic team mapping from fixtures: {team_mapping_df.shape}")
else:
    print(f"\nFinal team mapping DataFrame shape: {team_mapping_df.shape}")
    print(f"Columns: {list(team_mapping_df.columns)}")

# Show the team mapping data
if team_mapping_df is not None:
    print(f"\n=== TEAM MAPPING OVERVIEW ===")
    print(f"Shape: {team_mapping_df.shape}")
    print(f"Columns: {list(team_mapping_df.columns)}")
    
    # Show sample data
    print(f"\nSample team mapping data:")
    display(team_mapping_df.head(10))
    
    # Check for team_id column
    if 'team_id' in team_mapping_df.columns:
        print(f"\n✓ team_id column found - perfect for joining!")
        unique_teams = team_mapping_df['team_id'].nunique()
        print(f"Unique team IDs: {unique_teams}")
    else:
        print(f"\n⚠️  No team_id column found. Available columns: {list(team_mapping_df.columns)}")
        
    # Check for team name column
    team_name_cols = [col for col in team_mapping_df.columns if 'name' in col.lower()]
    if team_name_cols:
        print(f"Team name columns: {team_name_cols}")
    else:
        print("No obvious team name column found")
else:
    print("Unable to load team mapping data")

=== LOADING TEAM MAPPING DATA ===

Trying Prod all teams: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\all_teams.json
Successfully loaded: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\all_teams.json
Successfully loaded team mapping: (1, 27)

Final team mapping DataFrame shape: (1, 27)
Columns: ['18bb7c10', '8602292d', '4ba7cbea', 'd07537b9', '943e8050', 'cff3d9bb', '47c64c55', 'd3fd31cc', 'a2d435b3', '822bd0ba', 'b8fd03ef', '19538871', 'b2b47a98', '1c781004', '1df6b87e', '33c895d4', '361ca564', '2abfe087', '7c21e445', '8cec06e1', 'fd962109', '5bfb9659', '60c6b05f', 'cd051869', 'e4a775cb', 'e297cd13', 'b74092de']

=== TEAM MAPPING OVERVIEW ===
Shape: (1, 27)
Columns: ['18bb7c10', '8602292d', '4ba7cbea', 'd07537b9', '943e8050', 'cff3d9bb', '47c64c55', 'd3fd31cc', 'a2d435b3', '822bd0ba', 'b8fd03ef', '19538871', 'b2b47a98', '1c781004', '1df6b87e', '33c895d4', '361ca564', '2abfe

,18bb7c10,8602292d,4ba7cbea,d07537b9,943e8050,cff3d9bb,47c64c55,d3fd31cc,a2d435b3,822bd0ba,...,2abfe087,7c21e445,8cec06e1,fd962109,5bfb9659,60c6b05f,cd051869,e4a775cb,e297cd13,b74092de
0,"{'team_name': 'Arsenal', 'team_id': '18bb7c10'...","{'team_name': 'Aston Villa', 'team_id': '86022...","{'team_name': 'Bournemouth', 'team_id': '4ba7c...","{'team_name': 'Brighton', 'team_id': 'd07537b9...","{'team_name': 'Burnley', 'team_id': '943e8050'...","{'team_name': 'Chelsea', 'team_id': 'cff3d9bb'...","{'team_name': 'Crystal Palace', 'team_id': '47...","{'team_name': 'Everton', 'team_id': 'd3fd31cc'...","{'team_name': 'Leicester City', 'team_id': 'a2...","{'team_name': 'Liverpool', 'team_id': '822bd0b...",...,"{'team_name': 'Watford', 'team_id': '2abfe087'...","{'team_name': 'West Ham', 'team_id': '7c21e445...","{'team_name': 'Wolves', 'team_id': '8cec06e1',...","{'team_name': 'Fulham', 'team_id': 'fd962109',...","{'team_name': 'Leeds United', 'team_id': '5bfb...","{'team_name': 'West Brom', 'team_id': '60c6b05...","{'team_name': 'Brentford', 'team_id': 'cd05186...","{'team_name': 'Nott'ham Forest', 'team_id': 'e...","{'team_name': 'Luton Town', 'team_id': 'e297cd...","{'team_name': 'Ipswich Town', 'team_id': 'b740..."



⚠️  No team_id column found. Available columns: ['18bb7c10', '8602292d', '4ba7cbea', 'd07537b9', '943e8050', 'cff3d9bb', '47c64c55', 'd3fd31cc', 'a2d435b3', '822bd0ba', 'b8fd03ef', '19538871', 'b2b47a98', '1c781004', '1df6b87e', '33c895d4', '361ca564', '2abfe087', '7c21e445', '8cec06e1', 'fd962109', '5bfb9659', '60c6b05f', 'cd051869', 'e4a775cb', 'e297cd13', 'b74092de']
No obvious team name column found


In [5]:
# Look for team mapping files
print("=== SEARCHING FOR TEAM MAPPING FILES ===")

# Check for team mapping files in both environments
team_mapping_keywords = ['team', 'mapping', 'teams']

print("\nDev environment:")
dev_team_files = find_data_files(data_dev_path, team_mapping_keywords)
for file in dev_team_files:
    print(f"  - {file}")

print("\nProd environment:")
prod_team_files = find_data_files(data_prod_path, team_mapping_keywords)
for file in prod_team_files:
    print(f"  - {file}")

# Also check for any files that might contain team information
potential_team_files = []

# Check both environments for files that might contain team mappings
for env_name, env_path in [("Dev", data_dev_path), ("Prod", data_prod_path)]:
    if env_path.exists():
        all_files = os.listdir(env_path)
        for file in all_files:
            if any(keyword in file.lower() for keyword in ['team', 'mapping', 'all_teams']):
                potential_team_files.append((env_name, file, env_path / file))

print(f"\n=== POTENTIAL TEAM MAPPING FILES ===")
for env, filename, filepath in potential_team_files:
    print(f"  {env}: {filename}")
    # Check file size to get an idea of content
    try:
        file_size = filepath.stat().st_size
        print(f"    Size: {file_size:,} bytes")
    except:
        print(f"    Size: unknown")

=== SEARCHING FOR TEAM MAPPING FILES ===

Dev environment:
  - all_teams.json

Prod environment:
  - all_teams.json

=== POTENTIAL TEAM MAPPING FILES ===
  Dev: all_teams.json
    Size: 6,546 bytes
  Prod: all_teams.json
    Size: 6,546 bytes


In [7]:
team_mapping_data

{'18bb7c10': {'team_name': 'Arsenal',
  'team_id': '18bb7c10',
  'seasons': ['2019-2020',
   '2020-2021',
   '2021-2022',
   '2022-2023',
   '2023-2024',
   '2024-2025'],
  'aliases': ['Arsenal']},
 '8602292d': {'team_name': 'Aston Villa',
  'team_id': '8602292d',
  'seasons': ['2019-2020',
   '2020-2021',
   '2021-2022',
   '2022-2023',
   '2023-2024',
   '2024-2025'],
  'aliases': ['Aston Villa']},
 '4ba7cbea': {'team_name': 'Bournemouth',
  'team_id': '4ba7cbea',
  'seasons': ['2019-2020', '2022-2023', '2023-2024', '2024-2025'],
  'aliases': ['Bournemouth']},
 'd07537b9': {'team_name': 'Brighton',
  'team_id': 'd07537b9',
  'seasons': ['2019-2020',
   '2020-2021',
   '2021-2022',
   '2022-2023',
   '2023-2024',
   '2024-2025'],
  'aliases': ['Brighton']},
 '943e8050': {'team_name': 'Burnley',
  'team_id': '943e8050',
  'seasons': ['2019-2020', '2020-2021', '2021-2022', '2023-2024'],
  'aliases': ['Burnley']},
 'cff3d9bb': {'team_name': 'Chelsea',
  'team_id': 'cff3d9bb',
  'seasons'

## Load Fixtures Data

Now let's load the fixtures data which contains match results and basic information.

In [ ]:
# Try to load fixtures data
fixtures_df = None

# Priority order for fixtures
potential_fixtures_files = [
    (data_prod_path / 'all_competitions_fixtures_dataframe.json', 'Prod all competitions dataframe'),
    (data_prod_path / 'all_competitions_fixtures.json', 'Prod all competitions'),
    (data_dev_path / 'all_competitions_fixtures_2019_2025.json', 'Dev all competitions 2019-2025'),
    (data_dev_path / 'all_competitions_fixtures_2019_2024.json', 'Dev all competitions 2019-2024'),
    (data_dev_path / 'premier_league_fixtures_2019_2024.json', 'Dev Premier League only'),
]

print("=== Loading Fixtures Data ===")
for file_path, description in potential_fixtures_files:
    if file_path.exists():
        print(f"\nTrying {description}: {file_path}")
        fixtures_data = load_json_data(file_path)
        
        if fixtures_data:
            # Check if it's a list or dict
            if isinstance(fixtures_data, list):
                fixtures_df = pd.DataFrame(fixtures_data)
            elif isinstance(fixtures_data, dict):
                # Could be nested structure, try to flatten
                try:
                    # If dict contains lists, try to extract them
                    if any(isinstance(v, list) for v in fixtures_data.values()):
                        # Find the largest list (likely the main data)
                        max_key = max(fixtures_data.keys(), key=lambda k: len(fixtures_data[k]) if isinstance(fixtures_data[k], list) else 0)
                        fixtures_df = pd.DataFrame(fixtures_data[max_key])
                    else:
                        fixtures_df = pd.DataFrame([fixtures_data])
                except Exception as e:
                    print(f"Error processing fixtures data: {e}")
                    continue
            
            if fixtures_df is not None and len(fixtures_df) > 0:
                print(f"Successfully loaded fixtures: {fixtures_df.shape}")
                break
            else:
                print(f"Empty or invalid fixtures data")
                fixtures_df = None
    else:
        print(f"File not found: {file_path}")

if fixtures_df is None:
    print("\nNo fixtures data could be loaded!")
else:
    print(f"\nFinal fixtures DataFrame shape: {fixtures_df.shape}")
    print(f"Columns: {list(fixtures_df.columns)}")

=== Loading Fixtures Data ===



Trying Prod all competitions dataframe: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\all_competitions_fixtures_dataframe.json
Successfully loaded: C:\Users\50230\OneDrive\Escritorio\Proyectos y trabajos\Personales\Pronósticos Football\data\prod\raw\all_competitions_fixtures_dataframe.json
Successfully loaded fixtures: (5751, 30)

Final fixtures DataFrame shape: (5751, 30)
Columns: ['team_name', 'season', 'team_id', 'date', 'comp', 'round', 'venue', 'result', 'opponent', 'attendance', 'captain', 'formation', 'referee', 'match_report', 'notes', 'date_href', 'start_time', 'comp_href', 'round_href', 'dayofweek', 'goals_for', 'goals_against', 'opponent_href', 'xg_for', 'xg_against', 'possession', 'captain_href', 'opp_formation', 'match_report_href', 'full_match_report_url']


## Data Overview and Exploration

Let's examine the structure and content of both datasets.

In [ ]:
print("=== MATCH STATISTICS DATA OVERVIEW ===")
if match_stats_df is not None:
    print(f"Shape: {match_stats_df.shape}")
    print(f"\nColumns ({len(match_stats_df.columns)}):")
    for i, col in enumerate(match_stats_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\nData types:")
    print(match_stats_df.dtypes)
    
    print(f"\nSample data:")
    display(match_stats_df.head(3))
    
    print(f"\nMissing values:")
    missing_stats = match_stats_df.isnull().sum()
    if missing_stats.sum() > 0:
        missing_pct = (missing_stats / len(match_stats_df)) * 100
        missing_df = pd.DataFrame({
            'Missing Count': missing_stats,
            'Missing %': missing_pct
        })
        display(missing_df[missing_df['Missing Count'] > 0].head(10))
    else:
        print("No missing values found")
else:
    print("No match statistics data available")

=== MATCH STATISTICS DATA OVERVIEW ===
Shape: (98164, 4)

Columns (4):
   1. match_id
   2. team_name
   3. stat_name
   4. stat_value

Data types:
match_id      object
team_name     object
stat_name     object
stat_value    object
dtype: object

Sample data:


,match_id,team_name,stat_name,stat_value
0,https://fbref.com/en/matches/1405a610/Newcastl...,Newcastle,Possession,38%
1,https://fbref.com/en/matches/1405a610/Newcastl...,Arsenal,Possession,62%
2,https://fbref.com/en/matches/1405a610/Newcastl...,Newcastle,Passing Accuracy,75%



Missing values:
No missing values found


In [ ]:
print("=== FIXTURES DATA OVERVIEW ===")
if fixtures_df is not None:
    print(f"Shape: {fixtures_df.shape}")
    print(f"\nColumns ({len(fixtures_df.columns)}):")
    for i, col in enumerate(fixtures_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\nData types:")
    print(fixtures_df.dtypes)
    
    print(f"\nSample data:")
    display(fixtures_df.head(3))
    
    print(f"\nMissing values:")
    missing_fixtures = fixtures_df.isnull().sum()
    if missing_fixtures.sum() > 0:
        missing_pct = (missing_fixtures / len(fixtures_df)) * 100
        missing_df = pd.DataFrame({
            'Missing Count': missing_fixtures,
            'Missing %': missing_pct
        })
        display(missing_df[missing_df['Missing Count'] > 0].head(10))
    else:
        print("No missing values found")
        
    # Show unique values for key categorical columns
    key_cols = ['competition', 'season'] if 'competition' in fixtures_df.columns and 'season' in fixtures_df.columns else []
    for col in key_cols:
        unique_vals = fixtures_df[col].unique()
        print(f"\nUnique {col} values ({len(unique_vals)}): {list(unique_vals)}")
else:
    print("No fixtures data available")

=== FIXTURES DATA OVERVIEW ===
Shape: (5751, 30)

Columns (30):
   1. team_name
   2. season
   3. team_id
   4. date
   5. comp
   6. round
   7. venue
   8. result
   9. opponent
  10. attendance
  11. captain
  12. formation
  13. referee
  14. match_report
  15. notes
  16. date_href
  17. start_time
  18. comp_href
  19. round_href
  20. dayofweek
  21. goals_for
  22. goals_against
  23. opponent_href
  24. xg_for
  25. xg_against
  26. possession
  27. captain_href
  28. opp_formation
  29. match_report_href
  30. full_match_report_url

Data types:
team_name                object
season                   object
team_id                  object
date                     object
comp                     object
round                    object
venue                    object
result                   object
opponent                 object
attendance               object
captain                  object
formation                object
referee                  object
match_report          

,team_name,season,team_id,date,comp,round,venue,result,opponent,attendance,...,goals_for,goals_against,opponent_href,xg_for,xg_against,possession,captain_href,opp_formation,match_report_href,full_match_report_url
0,Arsenal,2019-2020,18bb7c10,2019-08-11,Premier League,Matchweek 1,Away,W,Newcastle Utd,"47,635",...,1,0,/en/squads/b2b47a98/2019-2020/Newcastle-United...,1.1,0.4,62,/en/players/e61b8aee/Granit-Xhaka,3-5-2,/en/matches/1405a610/Newcastle-United-Arsenal-...,https://fbref.com/en/matches/1405a610/Newcastl...
1,Arsenal,2019-2020,18bb7c10,2019-08-17,Premier League,Matchweek 2,Home,W,Burnley,"60,214",...,2,1,/en/squads/943e8050/2019-2020/Burnley-Stats,0.8,1.5,67,/en/players/d4cb83cc/Nacho-Monreal,4-4-2,/en/matches/ff7eda21/Arsenal-Burnley-August-17...,https://fbref.com/en/matches/ff7eda21/Arsenal-...
2,Arsenal,2019-2020,18bb7c10,2019-08-24,Premier League,Matchweek 3,Away,L,Liverpool,"53,298",...,1,3,/en/squads/822bd0ba/2019-2020/Liverpool-Stats,1.0,2.5,48,/en/players/e61b8aee/Granit-Xhaka,4-3-3,/en/matches/102b241e/Liverpool-Arsenal-August-...,https://fbref.com/en/matches/102b241e/Liverpoo...



Missing values:


,Missing Count,Missing %
attendance,1100,19.127108
captain,2,0.034777
formation,2,0.034777
referee,6,0.104330
notes,5400,93.896714
xg_for,737,12.815163
xg_against,737,12.815163
possession,40,0.695531
captain_href,2,0.034777
opp_formation,2,0.034777


## Identify Common Keys for Joining

Let's identify the common columns that can be used to join match stats and fixtures data.

In [ ]:
print("=== IDENTIFYING JOIN KEYS ===")

if match_stats_df is not None and fixtures_df is not None:
    stats_cols = set(match_stats_df.columns)
    fixtures_cols = set(fixtures_df.columns)
    
    common_cols = stats_cols.intersection(fixtures_cols)
    
    print(f"Common columns ({len(common_cols)}): {sorted(common_cols)}")
    
    # Look for potential key columns
    potential_keys = ['match_id', 'fixture_id', 'id', 'date', 'home_team', 'away_team', 'season']
    
    print("\n=== Potential Join Keys ===")
    for key in potential_keys:
        in_stats = key in stats_cols
        in_fixtures = key in fixtures_cols
        
        if in_stats and in_fixtures:
            print(f"✓ {key}: Present in both datasets")
            
            # Check uniqueness
            stats_unique = match_stats_df[key].nunique()
            fixtures_unique = fixtures_df[key].nunique()
            stats_total = len(match_stats_df)
            fixtures_total = len(fixtures_df)
            
            print(f"  Stats: {stats_unique}/{stats_total} unique values ({stats_unique/stats_total*100:.1f}% unique)")
            print(f"  Fixtures: {fixtures_unique}/{fixtures_total} unique values ({fixtures_unique/fixtures_total*100:.1f}% unique)")
            
            # Sample values
            if match_stats_df[key].dtype == 'object':
                sample_stats = list(match_stats_df[key].dropna().unique()[:3])
                sample_fixtures = list(fixtures_df[key].dropna().unique()[:3])
                print(f"  Sample stats values: {sample_stats}")
                print(f"  Sample fixtures values: {sample_fixtures}")
            
        elif in_stats:
            print(f"- {key}: Only in match stats")
        elif in_fixtures:
            print(f"- {key}: Only in fixtures")
        
        print()
    
    # Check for team-related columns
    print("=== Team-related columns ===")
    team_keywords = ['team', 'home', 'away', 'opponent']
    
    for dataset_name, df in [('Match Stats', match_stats_df), ('Fixtures', fixtures_df)]:
        team_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in team_keywords)]
        print(f"{dataset_name}: {team_cols}")

else:
    print("Cannot identify join keys - one or both datasets are missing")

=== IDENTIFYING JOIN KEYS ===
Common columns (1): ['team_name']

=== Potential Join Keys ===
- match_id: Only in match stats



- date: Only in fixtures



- season: Only in fixtures

=== Team-related columns ===
Match Stats: ['team_name']
Fixtures: ['team_name', 'team_id', 'opponent', 'opponent_href']


In [ ]:
team_mapping_df

NameError: name 'team_mapping_data' is not defined

In [10]:
match_stats_df

,match_id,team_name,stat_name,stat_value
0,https://fbref.com/en/matches/1405a610/Newcastl...,Newcastle,Possession,38%
1,https://fbref.com/en/matches/1405a610/Newcastl...,Arsenal,Possession,62%
2,https://fbref.com/en/matches/1405a610/Newcastl...,Newcastle,Passing Accuracy,75%
3,https://fbref.com/en/matches/1405a610/Newcastl...,Arsenal,Passing Accuracy,84%
4,https://fbref.com/en/matches/1405a610/Newcastl...,Newcastle,Shots on Target,22%
...,...,...,...,...
98159,https://fbref.com/en/matches/be42686a/Coventry...,Town,Crosses,9
98160,https://fbref.com/en/matches/be42686a/Coventry...,Coventry,Interceptions,13
98161,https://fbref.com/en/matches/be42686a/Coventry...,Town,Interceptions,14
98162,https://fbref.com/en/matches/be42686a/Coventry...,Coventry,Offsides,3


In [11]:
fixtures_df['team_name']

0            Arsenal
1            Arsenal
2            Arsenal
3            Arsenal
4            Arsenal
            ...     
5746    Ipswich Town
5747    Ipswich Town
5748    Ipswich Town
5749    Ipswich Town
5750    Ipswich Town
Name: team_name, Length: 5751, dtype: object

In [12]:
a = fixtures_df.merge(
    match_stats_df,
    left_on = 'full_match_report_url',
    right_on = 'match_id',
    how = 'left'
)

In [2]:
a[a['team_name_x']=='Manchester City']

NameError: name 'a' is not defined

## Team Name Inconsistency Analysis

Let's analyze the team name inconsistencies we've discovered and create a solution."

In [ ]:
print("=== TEAM NAME INCONSISTENCY ANALYSIS ===")

# Get unique team names from both datasets
fixtures_teams = set(fixtures_df['team_name'].unique())
stats_teams = set(match_stats_df['team_name'].unique())

print(f"Unique teams in fixtures: {len(fixtures_teams)}")
print(f"Unique teams in match stats: {len(stats_teams)}")

# Find teams that appear in one dataset but not the other
fixtures_only = fixtures_teams - stats_teams
stats_only = stats_teams - fixtures_teams
common_teams = fixtures_teams.intersection(stats_teams)

print(f"\nTeams in both datasets: {len(common_teams)}")
print(f"Teams only in fixtures: {len(fixtures_only)}")
print(f"Teams only in match stats: {len(stats_only)}")

print(f"\n=== TEAMS ONLY IN FIXTURES ===")
for team in sorted(fixtures_only):
    print(f"  - {team}")

print(f"\n=== TEAMS ONLY IN MATCH STATS ===")
for team in sorted(stats_only):
    print(f"  - {team}")

# Look for potential matches based on partial names
print(f"\n=== POTENTIAL MATCHES (Teams that might be the same) ===")
for fixtures_team in sorted(fixtures_only):
    for stats_team in sorted(stats_only):
        # Check if one name is contained in the other or if they share common words
        fixtures_words = set(fixtures_team.lower().split())
        stats_words = set(stats_team.lower().split())
        
        # If they share any significant words (not common words like 'fc', 'united', etc.)
        common_words = fixtures_words.intersection(stats_words)
        if common_words and not common_words.issubset({'fc', 'united', 'city', 'town'}):
            print(f"  '{fixtures_team}' <-> '{stats_team}' (common: {common_words})")
        
        # Check if stats team name appears in fixtures team name or vice versa
        elif (stats_team.lower() in fixtures_team.lower() or 
              fixtures_team.lower() in stats_team.lower()):
            print(f"  '{fixtures_team}' <-> '{stats_team}' (substring match)")

In [ ]:
## Create Improved Master Table

Now let's create the master table using our standardized team names."

In [ ]:
def create_team_name_mapping():
    """
    Create a comprehensive team name mapping to standardize names across datasets.
    Handles cases like 'Town' -> 'Ipswich Town', 'City' -> 'Manchester City', etc.
    """
    
    # Step 1: Create base mapping for known problematic cases
    team_name_mapping = {
        # Handle "Town" cases - we need to determine which "Town" refers to which team
        # This will be determined by context analysis
        
        # Handle "City" cases
        'City': 'Manchester City',  # Most common "City" in Premier League
        
        # Handle common abbreviations and variations
        'Man City': 'Manchester City',
        'Man Utd': 'Manchester United',
        'Spurs': 'Tottenham',
        'Tottenham Hotspur': 'Tottenham',
        'Leicester': 'Leicester City',
        'Brighton & Hove Albion': 'Brighton',
        'Brighton and Hove Albion': 'Brighton',
        'Wolves': 'Wolverhampton',
        'Wolverhampton Wanderers': 'Wolverhampton',
        'Sheffield United': 'Sheffield Utd',
        'Newcastle United': 'Newcastle Utd',
        'West Ham United': 'West Ham',
        'Nottingham Forest': "Nott'ham Forest",
        'Norwich City': 'Norwich',
    }
    
    return team_name_mapping

def analyze_ambiguous_names(fixtures_df, match_stats_df):
    """
    Analyze ambiguous team names like 'Town' and determine which team they refer to
    by looking at the context (opponents, dates, etc.)
    """
    
    # Get all instances where team name is just "Town"
    town_matches_stats = match_stats_df[match_stats_df['team_name'] == 'Town']['match_id'].unique()
    
    print(f"Found {len(town_matches_stats)} matches with 'Town' in match stats")
    
    # For each match, find what the opponent teams are in fixtures
    town_context = []
    
    for match_id in town_matches_stats[:10]:  # Check first 10 for analysis
        # Find the corresponding fixture
        fixture_match = fixtures_df[fixtures_df['full_match_report_url'] == match_id]
        
        if not fixture_match.empty:
            fixture_info = fixture_match.iloc[0]
            town_context.append({
                'match_id': match_id,
                'fixture_team': fixture_info['team_name'],
                'opponent': fixture_info['opponent'],
                'date': fixture_info['date'],
                'season': fixture_info['season']
            })
    
    print("\n=== CONTEXT ANALYSIS FOR 'Town' ===\n")
    for context in town_context:
        print(f"Match: {context['fixture_team']} vs {context['opponent']} ({context['date']})")
        print(f"  -> 'Town' likely refers to: {context['fixture_team']}")
        print()
    
    return town_context

def create_smart_team_mapping(fixtures_df, match_stats_df):
    """
    Create an intelligent team mapping based on context analysis
    """
    
    # Start with base mapping
    mapping = create_team_name_mapping()
    
    # Analyze ambiguous cases
    print("=== ANALYZING AMBIGUOUS TEAM NAMES ===")
    
    # Analyze "Town" cases
    town_context = analyze_ambiguous_names(fixtures_df, match_stats_df)
    
    # Based on the context analysis, create specific mappings
    # We'll determine this dynamically based on which teams appear with "Town"
    
    # Get all teams from fixtures that contain "Town"
    town_teams = [team for team in fixtures_df['team_name'].unique() if 'Town' in team]
    print(f"\nTeams containing 'Town' in fixtures: {town_teams}")
    
    # For each "Town" context, map to the appropriate full team name
    town_mapping = {}
    for context in town_context:
        fixture_team = context['fixture_team']
        if 'Town' in fixture_team:
            town_mapping[fixture_team] = fixture_team  # Use the full name from fixtures
    
    # If we find that "Town" in stats corresponds to specific teams, add those mappings
    if town_mapping:
        print(f"\nDynamic Town mappings detected:")
        for short_name, full_name in town_mapping.items():
            if 'Ipswich' in full_name:
                mapping['Town'] = 'Ipswich Town'
                print(f"  'Town' -> 'Ipswich Town' (based on context)")
            elif 'Luton' in full_name:
                mapping['Town'] = 'Luton Town'
                print(f"  'Town' -> 'Luton Town' (based on context)")
    
    return mapping

# Create the smart mapping
team_mapping = create_smart_team_mapping(fixtures_df, match_stats_df)

print(f"\n=== FINAL TEAM NAME MAPPING ===")
for short_name, full_name in team_mapping.items():
    print(f"  '{short_name}' -> '{full_name}'")

In [ ]:
## Apply Team Name Standardization

Now let's apply the mapping to clean our datasets and create a proper master table."

In [ ]:
# Convert nested JSON structure to DataFrame
if team_mapping_data and isinstance(team_mapping_data, dict):
    print("=== CONVERTING NESTED JSON TO DATAFRAME ===")
    
    # Method 1: Simple approach - flatten the nested structure
    team_records = []
    
    for team_id, team_info in team_mapping_data.items():
        # Create a record for each team
        record = {
            'team_id': team_id,
            'team_name': team_info.get('team_name', ''),
            'seasons': team_info.get('seasons', []),
            'aliases': team_info.get('aliases', [])
        }
        team_records.append(record)
    
    # Create DataFrame
    team_mapping_df = pd.DataFrame(team_records)
    
    print(f"Successfully converted to DataFrame: {team_mapping_df.shape}")
    print(f"Columns: {list(team_mapping_df.columns)}")
    
    # Show sample
    print("\n=== CONVERTED TEAM MAPPING DATA ===")
    display(team_mapping_df.head())
    
    # Handle list columns (seasons and aliases) if needed
    print(f"\n=== COLUMN DETAILS ===")
    print(f"team_id: {team_mapping_df['team_id'].dtype} - {team_mapping_df['team_id'].nunique()} unique values")
    print(f"team_name: {team_mapping_df['team_name'].dtype} - {team_mapping_df['team_name'].nunique()} unique values")
    print(f"seasons: contains lists - sample: {team_mapping_df['seasons'].iloc[0] if len(team_mapping_df) > 0 else 'empty'}")
    print(f"aliases: contains lists - sample: {team_mapping_df['aliases'].iloc[0] if len(team_mapping_df) > 0 else 'empty'}")
    
    # If you need to work with seasons or aliases, you might want to explode them
    print(f"\n=== OPTIONS FOR LIST COLUMNS ===")
    print("If you need individual rows for each season:")
    print("  team_mapping_exploded = team_mapping_df.explode('seasons')")
    print("\nIf you need individual rows for each alias:")
    print("  team_mapping_exploded = team_mapping_df.explode('aliases')")
    print("\nFor your use case (joining on team_id), the current format should work perfectly!")

else:
    print("team_mapping_data is not in the expected nested dictionary format")

In [ ]:
## Smart Team Name Standardization Solution

Let's create a comprehensive solution to handle the team name inconsistencies, including the \"Town\" problem."

In [ ]:
# Create improved master table with standardized team names
print("=== CREATING IMPROVED MASTER TABLE ===")

# Join using standardized team names
master_df_improved = pd.merge(
    fixtures_clean,
    match_stats_clean,
    left_on=['full_match_report_url'],
    right_on=['match_id'],
    how='left',
    suffixes=('_fixture', '_stats')
)

print(f"Master table shape: {master_df_improved.shape}")
print(f"Total columns: {len(master_df_improved.columns)}")

# Check how many matches now have statistics
matches_with_stats = master_df_improved['team_name_standardized_stats'].notna().sum()
total_fixture_rows = len(master_df_improved)

print(f"\nMatches with statistics: {matches_with_stats}/{total_fixture_rows} ({matches_with_stats/total_fixture_rows*100:.1f}%)")

# Verify team name consistency
print("\n=== TEAM NAME CONSISTENCY CHECK ===")

# Check cases where fixture and stats team names don't match (they should now!)
inconsistent_teams = master_df_improved[
    (master_df_improved['team_name_standardized_fixture'] != master_df_improved['team_name_standardized_stats']) &
    (master_df_improved['team_name_standardized_stats'].notna())
]

if len(inconsistent_teams) > 0:
    print(f"WARNING: Found {len(inconsistent_teams)} rows with inconsistent team names:")
    team_inconsistencies = inconsistent_teams[[
        'team_name_standardized_fixture', 
        'team_name_standardized_stats'
    ]].drop_duplicates()
    display(team_inconsistencies.head(10))
else:
    print("✓ All team names are now consistent between fixtures and stats!")

# Show sample of the improved master table
print("\n=== SAMPLE OF IMPROVED MASTER TABLE ===")
sample_cols = ['team_name_standardized_fixture', 'opponent', 'date', 'result', 
               'team_name_standardized_stats', 'stat_name', 'stat_value']
available_cols = [col for col in sample_cols if col in master_df_improved.columns]

display(master_df_improved[available_cols].head(10))